In [1]:
import pandas as pd
import html5lib
import requests as rq
from bs4 import BeautifulSoup
import os
import pathlib

def cols(d):
    print(list(d.columns))


cpath = os.getcwd() + '/../data/'
ppath = str(pathlib.Path(cpath))

### * 選挙概要

### * 選挙結果

In [ ]:
cols = ['当落','得票数','a','氏名','年齢','性別','党派','新旧','主な肩書き','b','区名','年']
sum_cols =['区名','年','告示日','投票日','定数・候補者数', '執行理由','有権者数','投票率','前回投票率','関連','b']
#urls = pd.read_excel(cpath + 'chiyoda.xlsx')
urls = pd.read_excel(cpath + 'ward_election_datasite.xlsx')

result = pd.DataFrame(columns=cols)

for url in urls.itertuples():
    try:
        r = rq.get(url.URL)
        soup = BeautifulSoup(r.content, "html.parser")
        tabular = soup.find("table", {'id':'smp-table-14040'}).find_all('tr')

        result_list = []
        for x in tabular[3:-3]:
            l = [td.text.replace('\u3000',' ').replace('\n','') for td in x.find_all('td')] + [url.ward, url.year]
            l[3] = x.find('a', {'class':'name-kanji'}).text.replace('\u3000',' ')
            if l[0] =='':
                l[0] = '落'
            result_list.append(l)

        result = pd.concat([result, pd.DataFrame(result_list, columns=cols)])
    except:
        print('Error@'+url.ward+str(url.year))


try:
    result.to_csv(cpath + 'election.csv', index=False)
except:
    print('Error-Write')



# 統合

### 1.読込み

In [2]:
cur_coln = ['年','区名','当落', '得票数', '氏名', '年齢', '性別', '党派', '新旧', '主な肩書き']
smy_coln = ['年', '区名', '投票日', '告示日', '定数', '候補者数', '有権者数','投票率%']
pre = pd.read_csv(cpath + 'election_org.csv')
pre['得票率%']=round(pre['得票率%'],2)

cur = pd.read_csv(cpath + 'all_election.csv')[cur_coln]
ward = pd.read_csv(cpath + 'ward_master.csv')
par = pd.read_csv(cpath + 'parties_org.csv')
smy = pd.read_csv(cpath + 'all_election_summary.csv')[smy_coln]
psmy = pd.read_csv(cpath + 'summary_org.csv')

### 2.概要

In [3]:
smy['投票者数'] = (smy['投票率%']/100 ) * smy['有権者数']

full_smy = pd.concat([psmy, smy])

full_smy.to_csv(cpath + 'summary.csv', index=False)
full_smy

,年,区名,投票日,告示日,定数,候補者数,有権者数,投票率%,投票者数
0,2011,目黒,2011/4/24,2011/4/17,35,55,"213,142",38.96,"83,040.123"
1,2011,品川,2011/4/24,2011/4/17,40,55,"296,585",40.58,"120,354.193"
2,2011,大田,2011/4/24,2011/4/17,50,73,"561,414",43.29,"243,036.121"
3,2011,渋谷,2011/4/24,2011/4/17,34,46,"168,344",40.22,"67,707.957"
4,2011,中央,2011/4/24,2011/4/17,30,39,"97,887",45.59,"44,626.683"
...,...,...,...,...,...,...,...,...,...
42,2015,江戸川,2015/4/26,2015/4/19,44,58,523417.0,43.12,225697.4104
43,2019,江戸川,2019/4/21,2019/4/14,44,58,547625.0,42.40,232193.0
44,2011,千代田,2011/4/24,2011/4/17,25,41,39195.0,49.66,19464.237
45,2015,千代田,2015/4/26,2015/4/19,25,38,44202.0,47.97,21203.6994


In [4]:
small_smy=smy[['年','区名','投票者数']] ##　結果整理用

### 3.党派

### 4.投票結果

In [5]:
dt = pd.merge(cur, small_smy, on=['年','区名']) # 選挙結果

dt=pd.merge(dt, par, on='党派', how='left')

dt.loc[1049,'党派']='不明'
dt.loc[1821,'党派']='不明'

dt['よみ']=''
dt['得票率%'] = round((dt['得票数']/dt['投票者数'])*100,2)

dttmp = pd.merge(dt, ward, on='区名')

final=pd.concat([pre,dttmp.drop(columns=['投票者数'])])

final.to_csv(cpath+'election.csv', index=False)

### 5. ward

In [6]:
rene = {'団体コード':'自治体コード', '団体名':'区名'}

tkcode = pd.read_csv(cpath + 'tokyo_code.csv')[['団体コード','団体名']].rename(columns=rene)


In [7]:
tkcode['区名']=tkcode['区名'].str.replace('区','')

In [8]:
tkcode.to_csv(cpath + 'tkcode.csv', index=False)

### 政治山Site

In [9]:
seijiyama = ['https://seijiyama.jp/area/table/3624/BjtDe5/M?detect=%E5%88%A4%E5%AE%9A&3734_252153_2=&3734_252153_1=&_limit_3624=200&S=qipe2lcqbo&3734_252159_1=2&3734_252150_1=&3734_252149_1=13&_page_3624=1',
            'https://seijiyama.jp/area/table/3624/BjtDe5/M?detect=%E5%88%A4%E5%AE%9A&3734_252153_2=&3734_252153_1=&_limit_3624=200&S=qipe2lcqbo&3734_252159_1=2&3734_252150_1=&3734_252149_1=13&_page_3624=2']

r = rq.get(seijiyama[0])
soup = BeautifulSoup(r.content, "html.parser")
tabular = soup.find_all("table")

#for sy in seijiyama:
#    r = rq.get(sy)
#    soup = BeautifulSoup(r.content, "html.parser")

In [10]:
list(map(lambda x: x.text,tabular[3].find_all('tr')[6].find_all('td')))

['2022-12-04', '2022-11-27', '品川区議会議員補欠選挙', '東京都', '品川区議補選 候補者へ', '']

In [11]:
tabular[3].find_all('tr')[6].find_all('td')

[<td align="center" class="smp-cell-data smp-cell-6-1 smp-cell smp-cell-row-6 smp-cell-col-1" style="border-width:0px 1px;border-style:solid;border-color:#999999;padding:2px;background-color:#f0f0f0;color:#444444;font-size:10pt;">2022-12-04</td>,
 <td align="center" class="smp-cell-data smp-cell-6-2 smp-cell smp-cell-row-6 smp-cell-col-2" style="border-width:0px 1px;border-style:solid;border-color:#999999;padding:2px;background-color:#f0f0f0;color:#444444;font-size:10pt;">2022-11-27</td>,
 <td align="left" class="smp-cell-data smp-cell-6-3 smp-cell smp-cell-row-6 smp-cell-col-3" style="border-width:0px 1px;border-style:solid;border-color:#999999 #999999 #999999 #ada296;padding:5px;background-color:#f0f0f0;color:#444444;font-size:10pt;">品川区議会議員補欠選挙</td>,
 <td align="center" class="smp-cell-data smp-cell-6-4 smp-cell smp-cell-row-6 smp-cell-col-4" style="border-width:0px 1px;border-style:solid;border-color:#999999 #999999 #999999 #ada296;padding:5px;background-color:#f0f0f0;color:#444444

In [12]:
'/area/card/3624/Cgh16a/M?S=lcqdt0lctgs0k'

'/area/card/3624/Cgh16a/M?S=lcqdt0lctgs0k'